In [1]:
import requests
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Setup splinter

executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\ccrom\.wdm\drivers\chromedriver\win32\99.0.4844.51]


## NASA MARS NEWS

In [3]:
# URL to visit
url = 'https://redplanetscience.com/'
browser.visit(url)

In [4]:
# Scrape the latest News Title & Paragraph Text 
html = browser.html
soup = bs(html, 'html.parser')

In [5]:
# Find news_title and news_p

news_title = soup.find('div', class_='content_title').text

news_p = soup.find('div', class_='article_teaser_body').text

print(f"Title: {news_title}")
print(f"Article: {news_p}")

Title: InSight's 'Mole' Team Peers into the Pit
Article: Efforts to save the heat probe continue.


## JPL Mars Space Image

In [6]:
# Visit new site to find image jpg. for feature image
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [7]:
# Use splint to click on FULL IMAGE button
browser.links.find_by_partial_text('FULL IMAGE').click()

In [8]:
# Scrape html
html = browser.html
soup = bs(html, 'html.parser')

In [9]:
# Store class with fancybox-inner in results
results = soup.find('div', class_= 'fancybox-inner')

In [10]:
# find image tag and index into img to find src
img = results.find('img')
link = img['src']

In [11]:
# get link to image and print it
featured_image_url = f'https://spaceimages-mars.com/{link}'
print(featured_image_url)

https://spaceimages-mars.com/image/featured/mars1.jpg


## Mars Facts

In [12]:
# Use pandas to scrape table from url
url = 'https://galaxyfacts-mars.com'

# read in tables, and print them
tables = pd.read_html(url)
tables

[                         0                1                2
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:          2 ( Phobos & Deimos )
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC

In [13]:
# Index into list to see the tables we have: Mars & Earth Comparison
tables[0]

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [14]:
# Index into second list item where we have Mars facts only (I prefer this one)
tables[1]

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 ( Phobos & Deimos )
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [15]:
# Save desired table to Variable df 
df = tables[1]
df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 ( Phobos & Deimos )
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [16]:
# Saving desired table to html and not including the header or index
df.to_html('marsfact_table.html', header = False, index = False, classes= 'table')

## Mars Hemispheres

In [17]:
# Visit new site to find image jpg. for Mars Hemisphere
url = 'https://marshemispheres.com/'
browser.visit(url)

In [18]:
# Scrape html and find all div, class descriptions
html = browser.html
soup = bs(html, 'html.parser')

results = soup.find_all('div', class_='description')

In [19]:
# Empty list to store titles
title_list = []

# Looping through results to find the titles, split, and append to list
for result in results:
    title_txt = result.find('h3').text
    title = title_txt.split('Enhanced')[0]
    title_list.append(title)

In [20]:
# List that will become a list of dictionaries
hemisphere_image_urls = []

# Using enumerate to go into titles list
for index, value in enumerate(title_list):
       
    # If statement to break out of loop when index is at 4 
    if index == 4:
        break
      
    # If the loop isn't ==4 then... get the value(title) to click into page
    else:
        url = 'https://marshemispheres.com/'
        browser.visit(url)  
        browser.links.find_by_partial_text(value).click()
        
        # Scrape html and grab download div
        html = browser.html
        soup = bs(html, 'html.parser')
        results = soup.find_all('div', class_='downloads')

        # Loop through the results to get image link in .jpg
        for result in results:
            u_list = result.find('ul')
            l_list = u_list.find('li')
            article = l_list.find('a')
            link = article['href']
            img_url = f'https://marshemispheres.com/{link}'

            # Store value and img_url as a dict
            img_dict = {
                'title' : value,
                'img_url' : img_url
            }

            # Append dictionary into hemisphere_image_urls list
            hemisphere_image_urls.append(img_dict)
            
            # Add one to index to move on 
            index+=1


print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere ', 'img_url': 'https://marshemispheres.com/images/full.jpg'}, {'title': 'Schiaparelli Hemisphere ', 'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg'}, {'title': 'Syrtis Major Hemisphere ', 'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg'}, {'title': 'Valles Marineris Hemisphere ', 'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg'}]


In [21]:
# Quit browser once scraping is complete
browser.quit()